In [1]:
from collections import Counter
import numpy as np
from pymatgen.io.vasp import Poscar
from potentials import BuckinghamPotential
from potential_parameters import BuckinghamParameter
from atom_types import AtomType

def types_from_structure(structure):
    elements = Counter(structure.species)
    for e in elements:
        print(e.name)
        print(type(e.name))


In [2]:
structure = Poscar.from_file('poscars/training_set/POSCAR1').structure

In [3]:
for site in structure:
    coords=site.coords

In [4]:
def pot_params():
    parameters = []
    parameters.append(BuckinghamParameter('Li_O_a', 'a', 1.0))
    parameters.append(BuckinghamParameter('Li_O_rho', 'rho', 0.1))
    parameters.append(BuckinghamParameter('Li_O_c', 'c', 0.0))
    return parameters

parameters = pot_params()
parameters

In [5]:
potentials = {'Li-O':[10.00, 0.1, 0.0]}
potentials

{'Li-O': [10.0, 0.1, 0.0]}

In [6]:
def atom_types():
    atom_types = []
    atom_types.append(AtomType(1, 'Li', 'Li', 1.555, 1.0))
    atom_types.append(AtomType(2, 'Ni', 'Ni', 56.111, 3.0))
    atom_types.append(AtomType(3, 'O core', 'O', 14.001, -0.96, 'core'))
    atom_types.append(AtomType(3, 'O shell', 'O', 1.998, -1.04, 'shell'))
    return atom_types

atom_type_list = atom_types()
atom_type_list

In [7]:
import mock

In [8]:
at = mock.Mock(AtomType)

In [9]:
at.label = "Li core"